In [1]:
!nvidia-smi

Sat Feb 27 02:47:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 450.57       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:18:00.0 Off |                  N/A |
| 35%   50C    P8    13W / 250W |      3MiB / 11016MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import wandb
import random
import os
from sklearn.model_selection import train_test_split
import collections
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
wandb.init(project = 'chest_xray')

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: wr80340 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
os.getcwd()

'/tf/liao'

In [4]:
training_path = []
training_labels = []
val_path = []
val_labels = []
testing_path = []
testing_labels = []

classes = ['NORMAL','PNEUMONIA']
for c in classes:  
    for dirpath, dnames, fnames in os.walk("./chest_xray/train/" + c):
        for f in fnames:
            training_path.append("./chest_xray/train/" + c + "/" + f)
            training_labels.append(c)
for c in classes:  
    for dirpath, dnames, fnames in os.walk("./chest_xray/val/" + c):
        for f in fnames:
            val_path.append("./chest_xray/val/" + c + "/" + f)
            val_labels.append(c)
for c in classes:  
    for dirpath, dnames, fnames in os.walk("./chest_xray/test/" + c):
        for f in fnames:
            testing_path.append("./chest_xray/test/" + c + "/" + f)
            testing_labels.append(c)
            
dt = list(zip(training_path, training_labels))
random.shuffle(dt)
training_path, training_labels = zip(*dt)
training_labels = list(training_labels)
training_path = list(training_path)
collections.Counter(training_labels)

Counter({'PNEUMONIA': 3875, 'NORMAL': 1341})

In [5]:
label_to_index = dict((name, index) for index,name in enumerate(classes))
index_to_label = dict((index, name) for index,name in enumerate(classes))
training_index = [label_to_index[lab] for lab in training_labels]
testing_index = [label_to_index[lab] for lab in testing_labels]
val_index = [label_to_index[lab] for lab in val_labels]
print(training_index[:5])
print(training_labels[:5])

[1, 0, 0, 1, 0]
['PNEUMONIA', 'NORMAL', 'NORMAL', 'PNEUMONIA', 'NORMAL']


In [6]:
training_index += val_index
training_path += val_path
training_path, val_path, training_index, val_index = train_test_split(training_path, training_index, test_size = 0.3, random_state = 1122)
print("training data : #{:0}, validation data #{:0}, testing data #{:0}".format(len(training_index), len(val_index), len(testing_index)))

training data : #3662, validation data #1570, testing data #624


In [7]:
IMAGE_SIZE_CROPPED = 200
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH = 224,224,3
BATCH_SIZE = 16
LR = 1e-5
EPOCH = 50
REPEATS = 3
last_layer_length = len(classes)

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
train_loss = tf.keras.metrics.Mean()
val_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

optimizer = tf.keras.optimizers.Adam(lr = LR)
scce = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)

In [8]:
def load_image(image_path, index):
    # convert path to img
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=IMAGE_DEPTH)
    img = tf.image.resize_with_pad(img, IMAGE_HEIGHT, IMAGE_WIDTH)
    img = tf.cast(img, tf.float32)
#     img = tf.keras.applications.resnet.preprocess_input(img)
    
    return img, index

def augment_function(image, index):
    # location mapping
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # color mapping
    trigger = tf.random.uniform([])
    if trigger < 0.1:
        image = tf.image.random_crop(image, [IMAGE_SIZE_CROPPED, IMAGE_SIZE_CROPPED, IMAGE_DEPTH])
    elif trigger < 0.3:
        image = tf.image.adjust_contrast(image, 3)
    elif trigger < 0.5:
        image = tf.image.adjust_saturation(image, 3)
    elif trigger < 0.7:
        image = tf.image.adjust_hue(image, 0.5)
        
    # assert size = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH)
    image = tf.image.resize_with_crop_or_pad(image, IMAGE_HEIGHT, IMAGE_WIDTH)
    
    return image, index

def standardize(image, index):
    image = tf.image.per_image_standardization(image)
    
    return image, index

In [9]:
dataset_train = tf.data.Dataset.from_tensor_slices((training_path, training_index))
dataset_train = dataset_train.repeat(REPEATS)
dataset_train = dataset_train.map(load_image)
# dataset_train = dataset_train.map(augment_function)
dataset_train = dataset_train.batch(BATCH_SIZE)
dataset_train = dataset_train.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

dataset_val = tf.data.Dataset.from_tensor_slices((val_path, val_index))
dataset_val = dataset_val.map(load_image)
dataset_val = dataset_val.batch(BATCH_SIZE)
dataset_val = dataset_val.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

dataset_test = tf.data.Dataset.from_tensor_slices((testing_path, testing_index))
dataset_test = dataset_test.map(load_image)
dataset_test = dataset_test.batch(BATCH_SIZE)
dataset_test = dataset_test.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

In [10]:
img_inputs = tf.keras.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH))
resnet = tf.keras.applications.ResNet101(include_top = False, weights = 'imagenet')
for layers in resnet.layers:
    layers.trainable = False
x = resnet(img_inputs)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(256, activation = "relu")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(64, activation = "relu")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)

out = tf.keras.layers.Dense(last_layer_length, activation = "softmax")(x)
model = tf.keras.Model(inputs=img_inputs, outputs=out)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet101 (Model)            multiple                  42658176  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               51380736  
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328

In [11]:
@tf.function
def train_step(image, index):
    with tf.GradientTape() as tape:
        preds = model(image)
        loss = scce(index, preds)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(index, preds)

@tf.function
def val_step(image, index):
    preds = model(image)
    loss = scce(index, preds)
    
    val_loss(loss)
    val_accuracy(index, preds) 
    
@tf.function
def test_step(image, index):
    preds = model(image)
    loss = scce(index, preds)
    
    test_loss(loss)
    test_accuracy(index, preds)

In [12]:
for epoch in range(EPOCH):    
    for (train_image, train_index), (val_image, val_index) in zip(dataset_train, dataset_val): 
        train_step(train_image, train_index)
        val_step(val_image, val_index)

    wandb.log({
        "training accuracy" : train_accuracy.result().numpy(),
        "training loss" : train_loss.result().numpy(),
        "validation accuracy" : val_accuracy.result().numpy(),
        "validation loss" : val_loss.result().numpy(),
    })
    if (epoch+1) % 5 == 0:
        tmp = "EPOCH {:0}, train accuracy {:.2f}, train loss {:.2f}, val accuracy {:.2f}, val loss {:.2f}"
        print(tmp.format(epoch+1, train_accuracy.result().numpy(), train_loss.result().numpy(), val_accuracy.result().numpy(), val_loss.result().numpy()))

train_loss.reset_states()
train_accuracy.reset_states()
val_loss.reset_states()
val_accuracy.reset_states()

EPOCH 5, train accuracy 0.98, train loss 0.91, val accuracy 0.96, val loss 1.96
EPOCH 10, train accuracy 0.99, train loss 0.46, val accuracy 0.96, val loss 1.73
EPOCH 15, train accuracy 0.99, train loss 0.31, val accuracy 0.97, val loss 1.65
EPOCH 20, train accuracy 0.99, train loss 0.23, val accuracy 0.97, val loss 1.61
EPOCH 25, train accuracy 1.00, train loss 0.19, val accuracy 0.97, val loss 1.59
EPOCH 30, train accuracy 1.00, train loss 0.16, val accuracy 0.97, val loss 1.58
EPOCH 35, train accuracy 1.00, train loss 0.13, val accuracy 0.97, val loss 1.58
EPOCH 40, train accuracy 1.00, train loss 0.12, val accuracy 0.97, val loss 1.58
EPOCH 45, train accuracy 1.00, train loss 0.10, val accuracy 0.97, val loss 1.58
EPOCH 50, train accuracy 1.00, train loss 0.09, val accuracy 0.97, val loss 1.59


In [13]:
for test_image, test_index in dataset_test: 
    test_step(test_image, test_index)
tmp = "test accuracy {:.2f}, test loss {:.2f}"
print(tmp.format(test_accuracy.result().numpy(), test_loss.result().numpy()))

test accuracy 0.80, test loss 25.75
